#### 1. Preparation

In [1]:
# Packages
import numpy as np
import matplotlib.pyplot

import geopandas as gpd
import shapely.geometry
import json

import pathlib
from geofabrics import processor

In [2]:
# Neccesary variables
resolution_func = 10
chunk_size_func = 100
processor_func = 4

h_crs = 2193
v_crs = 7839

# Direct the basepath to the folder that contains generated 'dem.nc'
# Here I copied the generated 'dem.nc' into roughness folder for easy noticing
# You can direct the basepath to the same basepath of dem
basepath = pathlib.Path(r"S:\Grid_Ori_bigdata_005")
lidar_dataset_name = "Wellington_2013"
result_folder = "dem_generation"

#### 2. Create necessary variables

In [3]:
# Create main folder
data_dir = basepath / result_folder
data_dir.mkdir(exist_ok=True)

In [4]:
# Padding box coordinates
padding_func = [1767790, 5471400, 1774510, 5475880]
# padding_func = [1767790, 5471398, 1774508.5, 5475880]

# Bounding box
x0 = padding_func[0]
y0 = padding_func[1]
x1 = padding_func[2]
y1 = padding_func[3]
catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
catchment = gpd.GeoSeries([catchment])
catchment = catchment.set_crs(h_crs)

# Write out to file
catchment.to_file(basepath / result_folder / "catchment_boundary.geojson", crs="EPSG:2193", driver="GeoJSON")

#### 3. Generate roughness

In [5]:
# Setup for instructions
# Roughness
instruction_json = {
        # roughness - output
        "output": {
            "crs": {
                "horizontal": h_crs,
                "vertical": v_crs
            },
            "grid_params":{
                "resolution": resolution_func
            }
        },
        
        # roughness - processing
        "processing": {
            "chunk_size": chunk_size_func,
            "number_of_cores": processor_func
        },
        
        # roughness - data_paths
        "data_paths": {
            "local_cache": str(basepath),
            "subfolder": result_folder,
            "catchment_boundary": "catchment_boundary.geojson",
            "result_dem": "dem.nc",
            "result_geofabric": "zo.nc"
        },
        
        # roughness - apis
        "apis": {
            "open_topography": {
                "Wellington_2013": {
                    "crs": {
                        "horizontal": 2193,
                        "vertical": 7839
                    }
                }
            },
            "linz": {
                "key": "857413f41ce446ed8961e2f1e960a24b",
                "land": {
                    "layers": [51559]
                }
            }
        },
        
        # roughness - general
        "general": {
            "set_dem_shoreline": True,
            "drop_offshore_lidar": False,
            "lidar_classifications_to_keep": [1, 2, 4, 9],
            "interpolation_method": "linear",
            "lidar_interpolation_method": "idw"
        },
        
        # roughness - rivers
        "drains": {
            "width": 5
        }
    }

In [6]:
# Write out instructions
with open(basepath / result_folder / "instructions.json", "w") as instruction:
    json.dump(instruction_json, instruction)

In [7]:
%%time
# Run
runner = processor.RoughnessLengthGenerator(instruction_json)
runner.run()

Check files in dataset Wellington_2013
<Client: 'tcp://127.0.0.1:52419' processes=4 threads=4, memory=127.73 GiB>
CPU times: total: 36.4 s
Wall time: 4min 33s
